### Working on the nnpz task

See: https://redmine.astro.unige.ch/issues/15518

Downloaded catalogs and filters from Jean's area into /data/dc3. Copied SEDs from Phosphoros quickstart.

Tried to run using the 10x catalogs, but there are duplicated IDs, so Phosphoros failed. It seems the 10x is just
the same as the other catalog, but each entry 10 times.

Luckily I didn't run with the full set, or it would have taken forever. In any case, if they are exactly identical,
just 10 times repeated (to be checked, I may be wrong), I may as well replicate the result 10 times.

Run Phosphoros GUI, configure filters, generate the parameter space using the Quickstart values, and run the
template fitting.
After a couple of hours and burning CPU at 100%, got an output.

Run my ph2ref script on the Phosphoros output to generate a reference sample under /data/dc3/ReferenceSample

In [3]:
import nnpz
ref = nnpz.ReferenceSample('/data/dc3/ReferenceSample/')
print(ref.size())

518216


Run NnpzBuildPhotometry over the just created RefernceSample
```bash
./NnpzBuildPhotometry --sample-dir "/data/dc3/ReferenceSample" --filters "/data/dc3/AuxiliaryData/Filters/Euclid" --out-type F_nu_uJy --out-file "/data/dc3/DC3.2.0_F_nu_uJy.fits"
```

Run in 263.36s. An excerpt:

In [12]:
from astropy.table import Table
photometry = Table.read('/data/dc3/DC3.2.0_F_nu_uJy.fits', hdu=1)
photometry[0:5]

ID,g,r,J,Y,i,H,z,u,vis
int64,float32,float32,float32,float32,float32,float32,float32,float32,float32
216439,5.94379e+15,1.26111e+16,3.30109e+16,2.93887e+16,1.75996e+16,3.7315e+16,2.42022e+16,1.27347e+15,1.44539e+16
218604,5.40076e+15,1.02661e+16,2.16418e+16,1.97281e+16,1.34469e+16,2.34639e+16,1.71872e+16,1.44529e+15,1.13922e+16
219020,5.28514e+15,9.73779e+15,1.95312e+16,1.79251e+16,1.25439e+16,2.09468e+16,1.57865e+16,1.47245e+15,1.0722e+16
219656,3.03014e+15,3.09651e+15,3.01943e+15,3.18958e+15,3.16121e+15,2.93183e+15,3.31571e+15,1.96949e+15,3.12577e+15
219657,3.97374e+15,5.89777e+15,1.21127e+16,1.09102e+16,7.29244e+15,1.34564e+16,9.34767e+15,1.75235e+15,6.42259e+15


Create a new config file, and use that one to run nnpz
```python
reference_sample_dir = '/data/dc3/ReferenceSample'
reference_sample_phot_file = '/data/dc3/DC3.2.0_F_nu_uJy.fits'
reference_sample_phot_filters = ['u', 'g', 'r', 'i', 'z', 'vis', 'Y', 'J', 'H']
out_mean_phot_filters = reference_sample_phot_filters

target_cat = '/data/dc3/Catalog/Cosmos/euclid_emulated_DC3.2.0_COSMOS15_test.fits'
target_filters = [
    ('u_obs', 'u_obs_err'),
    ('g_obs', 'g_obs_err'),
    ('r_obs', 'r_obs_err'),
    ('i_obs', 'i_obs_err'),
    ('z_obs', 'z_obs_err'),
    ('vis_obs', 'vis_obs_err'),
    ('Y_obs', 'Y_obs_err'),
    ('J_obs', 'J_obs_err'),
    ('H_obs', 'H_obs_err')
]
missing_photometry_flags = [-99]

neighbor_method = 'Combined'

batch_size = 2000
neighbors_no = 30

input_size = 1000

output_file = '/data/dc3/nnpz/out.fits'
```

When running, this line takes a long time, and no feedback is available on the terminal :(
```python
selector = EuclideanRegionBruteForceSelector(neighbors_no, batch_size).initialize(ref_data)
```

Regenerated the photometries but with F_lambda, not sure if that's ok, need to ask. The values from the
previous one were too far away.

In [13]:
photometry_lambda = Table.read('/data/dc3/DC3.2.0_F_lambda.fits', hdu=1)
photometry_lambda[0:5]

ID,g,r,J,Y,i,H,z,u,vis
int64,float32,float32,float32,float32,float32,float32,float32,float32,float32
216439,0.00780618,0.0092537,0.00540401,0.00758704,0.00872465,0.00363316,0.00871057,0.00284716,0.00902161
218604,0.00709301,0.00753299,0.00354284,0.00509306,0.00666605,0.00228456,0.0061858,0.00323129,0.00711058
219020,0.00694116,0.00714534,0.00319733,0.00462758,0.00621838,0.00203948,0.00568171,0.00329203,0.00669232
219656,0.00397959,0.00227214,0.000494291,0.00082343,0.0015671,0.000285457,0.00119335,0.00440326,0.00195099
219657,0.00521885,0.00432763,0.0019829,0.00281661,0.00361508,0.00131018,0.0033643,0.0039178,0.00400876
